In [ ]:
import os
print(os.listdir("../input"))

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_csv('../input/sms-spam-collection-dataset/spam.csv', encoding='latin-1')[['v1', 'v2']]
df.columns = ['label', 'message']
df.head()

In [ ]:
df.groupby('label').describe()

In [ ]:
sns.countplot(data=df, x='label')

In [ ]:
import string
from nltk.corpus import stopwords
from nltk import PorterStemmer as Stemmer
def process(text):
    # lowercase it
    text = text.lower()
    # remove punctuation
    text = ''.join([t for t in text if t not in string.punctuation])
    # remove stopwords
    text = [t for t in text.split() if t not in stopwords.words('english')]
    # stemming
    st = Stemmer()
    text = [st.stem(t) for t in text]
    # return token list
    return text

In [ ]:
# Testing
process('It\'s holiday and we are playing cricket. Jeff is playing very well!!!')

In [ ]:
# Test with our dataset
df['message'][:20].apply(process)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidfv = TfidfVectorizer(analyzer=process)
data = tfidfv.fit_transform(df['message'])

In [ ]:
mess = df.iloc[2]['message']
print(mess)

In [ ]:
print(tfidfv.transform([mess]))

In [ ]:
j = tfidfv.transform([mess]).toarray()[0]
print('index\tidf\ttfidf\tterm')
for i in range(len(j)):
    if j[i] != 0:
        print(i, format(tfidfv.idf_[i], '.4f'), format(j[i], '.4f'), tfidfv.get_feature_names_out()[i],sep='\t')


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
spam_filter = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer=process)), # messages to weighted TFIDF score
    ('classifier', MultinomialNB())                    # train on TFIDF vectors with Naive Bayes
])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['message'], df['label'], test_size=0.3, random_state = 42)

In [ ]:
spam_filter.fit(x_train, y_train)

In [ ]:
predictions = spam_filter.predict(x_test)

In [ ]:
count = 0
for i in range(len(y_test)):
    if y_test.iloc[i] != predictions[i]:
        count += 1
print('Total number of test cases', len(y_test))
print('Number of wrong of predictions', count)

In [ ]:
x_test[y_test != predictions]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(predictions, y_test))

In [ ]:
def detect_spam(s):
    return spam_filter.predict([s])[0]
detect_spam('Win 500 pounds by scanning the QR code sent to you.')